In [1]:
import pandas as pd
import numpy as np

In [2]:
rating_df=pd.read_csv('RCdata/rating_final.csv')
cursine_df=pd.read_csv('RCdata/chefmozcuisine.csv')
places_df= pd.read_csv('RCdata/geoplaces2.csv', encoding ="ISO-8859-1")
print(rating_df.head(3))
print(cursine_df.head(3))
# print(geo_place_df.head(3))

  userID  placeID  rating  food_rating  service_rating
0  U1077   135085       2            2               2
1  U1077   135038       2            2               1
2  U1077   132825       2            2               2
   placeID        Rcuisine
0   135110         Spanish
1   135109         Italian
2   135107  Latin_American


In [3]:
print(places_df.head(1))

   placeID   latitude  longitude  \
0   134999  18.915421 -99.184871   

                                      the_geom_meter             name  \
0  0101000020957F000088568DE356715AC138C0A525FC46...  Kiku Cuernavaca   

      address        city    state country fax      ...        \
0  Revolucion  Cuernavaca  Morelos  Mexico   ?      ...         

             alcohol smoking_area dress_code     accessibility   price  \
0  No_Alcohol_Served         none   informal  no_accessibility  medium   

                     url Rambience franchise    area other_services  
0  kikucuernavaca.com.mx  familiar         f  closed           none  

[1 rows x 21 columns]


In [4]:
len(rating_df['placeID'].unique())

130

### Popularity-based recommender (based on counts)

In [179]:
pop_place_rank=pd.DataFrame(rating_df.groupby('placeID')['rating'].count()).sort_values('rating',ascending=False)

In [180]:
pop_rank=pd.merge(pop_place_rank, cursine_df, on='placeID')
pop_rank=pd.merge(pop_rank, places_df, on ='placeID')[['placeID', 'name', 'Rcuisine']]
pop_rank['Rcuisine']=pop_rank.groupby(['placeID', 'name'])['Rcuisine'].transform(lambda x: ','.join(x))
pop_rank=pop_rank.drop_duplicates()
pop_rank.head()

,placeID,name,Rcuisine
0,135085,Tortas Locas Hipocampo,Fast_Food
1,132825,puesto de tacos,Mexican
2,135032,Cafeteria y Restaurant El Pacifico,"Cafeteria,Contemporary"
4,135052,La Cantina Restaurante,"Bar,Bar_Pub_Brewery"
6,132834,Gorditas Doa Gloria,Mexican


In [11]:
cursine_df['Rcuisine'].describe()

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

The mexican food are most common and popular food type in this town, and among a large amout of Mexican food resturant, our popularity-based recommender will give ID 132825 and 132834 as the recommendations.

### Correlation-based Recommender (the correlation between users' review scores)

In [12]:
cor_rating=pd.DataFrame(rating_df.groupby('placeID')['rating'].mean())

In [13]:
cor_rating['rating_count']= rating_df.groupby('placeID')['rating'].count()
cor_rating=cor_rating.sort_values('rating_count', ascending=False)
cor_rating.head(3)

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28


In [14]:
print("the resturant's name is {}".format(list(places_df[places_df['placeID']==135085]['name'])[0]))
print("the resturant serves {} food".format(list(cursine_df[cursine_df['placeID']==135085]['Rcuisine'])[0]))

the resturant's name is Tortas Locas Hipocampo
the resturant serves Fast_Food food


Assuming there's an customer who just gives high rating to this most popular fast food resturant, Tortas Locas

In [15]:
## for personalization, we need a userID by placeID matrix. The values inside would the rating
user_by_place_rating = pd.pivot_table(rating_df, index='userID', columns= 'placeID', values= 'rating')
user_by_place_rating.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
print("""{:.2f}% of entries are not null, so it's a pretty sparse matrix""".format(np.sum(user_by_place_rating.count())/(user_by_place_rating.shape[0]*user_by_place_rating.shape[1])*100))

6.47% of entries are not null, so it's a pretty sparse matrix


#### Assuming someone went to "Tortas Locas Hipocampo", and we wanna send some recommendations

In [16]:
Tortas_rating=user_by_place_rating[135085]
similarity_to_Tortas=user_by_place_rating.corrwith(Tortas_rating)  ##compute Pearson R correlation values (from 1 to -1)
similarity_to_Tortas=similarity_to_Tortas.dropna().reset_index()
similarity_to_Tortas.rename(columns={0:'Pearson R'}, inplace=True)

C:\Users\ben29\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2392: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\ben29\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [17]:
similarity_to_Tortas=similarity_to_Tortas.sort_values('Pearson R', ascending=False)
similarity_to_Tortas.head(5)

,placeID,Pearson R
41,135076,1.0
10,132922,1.0
43,135085,1.0
12,132937,1.0
11,132925,1.0


In [18]:
##Explain how corrwith is working with lists which have null values in different positions
from scipy import stats
a=user_by_place_rating[132572].dropna().reset_index()
b=user_by_place_rating[135085].dropna().reset_index()
k=pd.merge(a,b, on ='userID')
stats.pearsonr(k[132572], k[135085])

(-0.42857142857142855, 0.2894032248467901)

In [19]:
## To avoid that the high correlation may only come form a small amount of review scores, we also needs to take popularity 
#  into account
similarity_to_Tortas=pd.merge(similarity_to_Tortas, cor_rating.reset_index(), on='placeID')

In [20]:
similarity_to_Tortas=similarity_to_Tortas[similarity_to_Tortas['rating_count']>=10].sort_values('Pearson R', ascending=False)
similarity_to_Tortas.head()

,placeID,Pearson R,rating,rating_count
0,135076,1.000000,1.307692,13
2,135085,1.000000,1.333333,36
5,135066,1.000000,1.500000,12
7,132754,0.930261,1.461538,13
8,135045,0.912871,1.461538,13


In [21]:
item_based_recommm=pd.merge(similarity_to_Tortas, cursine_df, on='placeID')[['placeID', 'Rcuisine']]
item_based_recommm=pd.merge(item_based_recommm, places_df, on='placeID')[['placeID', 'name', 'Rcuisine']]
item_based_recommm.head()  ##here are the recommendations.

,placeID,name,Rcuisine
0,135085,Tortas Locas Hipocampo,Fast_Food
1,132754,Cabana Huasteca,Mexican
2,135028,La Virreina,Mexican
3,135042,Restaurant Oriental Express,Chinese
4,135046,Restaurante El Reyecito,Fast_Food


In [151]:
## another way of correlation-based recommendation is to add some dimensionality reduction on the sparse rating matrix.
## the benefits of doing this is to include null values impact as well.
## I'll use SVD as the dimensionality reduction method in this example, since it's simple.
user_by_place_rating_svd = pd.pivot_table(rating_df, index='userID', columns= 'placeID', values= 'rating', fill_value=-1)
np.asarray(user_by_place_rating_svd).T.shape  ## 130 restaurants; 138 users
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100)
X_reduced = svd.fit_transform(np.asarray(user_by_place_rating_svd).T)
X_reduced.shape

(130, 100)

In [187]:
place_corr_matrix= np.corrcoef(X_reduced)
corr_with_tortas=place_corr_matrix[list(user_by_place_rating.columns).index(135085)]
corr_to_tortas_df=pd.DataFrame({'placeID': user_by_place_rating.columns, 'corr_to_tortas': corr_with_tortas})
corr_to_tortas_rank=corr_to_tortas_df[corr_to_tortas_df['corr_to_tortas']>0.45].sort_values(by=['corr_to_tortas'], ascending=False).reset_index(drop=True)
corr_to_tortas_rank

,placeID,corr_to_tortas
0,135085,1.000000
1,132825,0.530051
2,135030,0.494811
3,132834,0.470210
4,132951,0.457857


In [188]:
corr_to_tortas_rank=pd.merge(corr_to_tortas_rank, cursine_df, on='placeID')
corr_to_tortas_rank=pd.merge(corr_to_tortas_rank, places_df, on ='placeID')[['placeID', 'name', 'Rcuisine']]
corr_to_tortas_rank['Rcuisine']=corr_to_tortas_rank.groupby(['placeID', 'name'])['Rcuisine'].transform(lambda x: ','.join(x))
corr_to_tortas_rank=corr_to_tortas_rank.drop_duplicates()
corr_to_tortas_rank  ###Here is the rank based on this SVD + correlation based recommendation if a user lik Tortas

,placeID,name,Rcuisine
0,135085,Tortas Locas Hipocampo,Fast_Food
1,132825,puesto de tacos,Mexican
2,135030,Preambulo Wifi Zone Cafe,"Cafe-Coffee_Shop,Cafeteria"
4,132834,Gorditas Doa Gloria,Mexican
5,132951,VIPS,American


### User-based recommendation (classification using logistic regression)

In [82]:
user_df= pd.read_csv('RCdata/userprofile.csv')

In [83]:
user_df['birth_year']=user_df['birth_year'].astype('category')
part1_df= user_df[['userID']]  ## we don't want change userID into dummy variables
part2_df= pd.get_dummies(user_df[user_df.columns.drop('userID')])
user_df=pd.concat([part1_df, part2_df.reindex(part1_df.index)], axis=1)
user_df.head()

,userID,latitude,longitude,weight,height,smoker_?,smoker_false,smoker_true,drink_level_abstemious,drink_level_casual drinker,...,color_green,color_orange,color_purple,color_red,color_white,color_yellow,budget_?,budget_high,budget_low,budget_medium
0,U1001,22.139997,-100.978803,69,1.77,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,U1002,22.150087,-100.983325,40,1.87,0,1,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,U1003,22.119847,-100.946527,60,1.69,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,U1004,18.867000,-99.183000,44,1.53,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,U1005,22.183477,-100.959891,65,1.69,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


Assuming we want to know if a new user will like Tortas Locas Hipocampo. The criterial of whether he likes it is if he will give rating 2 to the resutaurant. 

In [84]:
Tortas_rating= rating_df[rating_df['placeID']==135085][['userID', 'rating']]  ##existing record of Tortas rating
log_classification_df= pd.merge(user_df, Tortas_rating, how='left', on='userID')
print('{} % of users gave rating to Totas resaurant'.format(log_classification_df.rating.describe()[0]/len(log_classification_df.rating)*100))
log_classification_df.head()

26.08695652173913 % of users gave rating to Totas resaurant


,userID,latitude,longitude,weight,height,smoker_?,smoker_false,smoker_true,drink_level_abstemious,drink_level_casual drinker,...,color_orange,color_purple,color_red,color_white,color_yellow,budget_?,budget_high,budget_low,budget_medium,rating
0,U1001,22.139997,-100.978803,69,1.77,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0.0
1,U1002,22.150087,-100.983325,40,1.87,0,1,0,1,0,...,0,0,1,0,0,0,0,1,0,1.0
2,U1003,22.119847,-100.946527,60,1.69,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
3,U1004,18.867000,-99.183000,44,1.53,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,NaN
4,U1005,22.183477,-100.959891,65,1.69,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,NaN


In [103]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
train_df= log_classification_df[log_classification_df['rating'].notnull()]
test_df=log_classification_df[log_classification_df['rating'].isnull()]
clf = LogisticRegression(random_state=0).fit(train_df.iloc[:, 1:84], train_df.iloc[:, 84:85])

C:\Users\ben29\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ben29\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ben29\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [124]:
result_df=test_df[['userID']]
result_df['pred_rating_tortas']= clf.predict(test_df.iloc[:, 1:84])

##we set up the recommendation criterial to be 
##1, the prediction rating has to be 2
##2, prediction probability of rating 2 >60% OR the sum of probabilities of rating 1 and rating 2 has to be >90%
criterial= [np.argmax(i)==2 and (i[2]>0.8 or i[1]+i[2]>0.99) for i in clf.predict_proba(test_df.iloc[:, 1:84])]
result_df=result_df.loc[criterial, :].reset_index(drop=True)   ##leave only the potential customer who satisfy our criterial
result_df

C:\Users\ben29\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userID,pred_rating_tortas
0,U1010,2.0
1,U1020,2.0
2,U1024,2.0
3,U1030,2.0
4,U1041,2.0
5,U1042,2.0
6,U1054,2.0
7,U1061,2.0
8,U1071,2.0
9,U1072,2.0


So based on our logistics model, we'd recommend Tortas restaurant to the 20 users above. 

### Content-based recommender using Nearest Neighbor

In [190]:
cursine_df.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [220]:
knn_recomm=pd.merge(places_df[['placeID','city', 'country', 'price']],cursine_df, how='right', on='placeID')
knn_recomm_df= pd.get_dummies(knn_recomm, columns=list(knn_recomm.columns.drop('placeID')))
X=knn_recomm_df.iloc[:,1:].values
tortas=X[np.where(knn_recomm_df["placeID"] == 135085)]

In [221]:
from sklearn.neighbors import NearestNeighbors
nbrs= NearestNeighbors(n_neighbors=5).fit(X)
nbrs_rank=nbrs.kneighbors(tortas)[1].squeeze()

In [222]:
nbrs_rank

array([ 36, 103,  10,   2,  12], dtype=int64)

In [225]:
pd.merge(knn_recomm.loc[nbrs_rank], places_df[['placeID', 'name']], on='placeID')
###that's the recommendation if someone gives high rating to Tortas Locas restaurant

,placeID,city,country,price,Rcuisine,name
0,135046,San Luis Potosi,Mexico,medium,Fast_Food,Restaurante El Reyecito
1,135085,San Luis Potosi,Mexico,medium,Fast_Food,Tortas Locas Hipocampo
2,135086,San Luis Potosi,Mexico,medium,Fast_Food,Mcdonalds Parque Tangamanga
3,135106,San Luis Potosi,Mexico,medium,Mexican,El Rincón de San Francisco
4,132854,San Luis Potosi,Mexico,medium,International,Sirlone
